В файле приведены как решения задач (в текстовых блоках), так и весь исполняемый код для решения задачи.

In [1]:
# загрузка файла по словам в список
text = []
def load(file_name):
    global text
    f = open(file_name, 'r')
    text = f.read().split()
    f.close()

In [2]:
# парсинг входного файла
class Event:
    def __init__(self, name):
        self.name = name

time = 0
def NextEvent():
    global time
    if time >= len(text):
        event = Event("the end")
    elif text[time] == 'book':
        event = Event("book")
        event.bids = []
        event.asks = []
        time += 1
        
        while True:
            if text[time] == "volumes:":
                time += 1
                while text[time] != '|':
                    event.bids.append(int(text[time]))
                    time += 1

                time += 1
                while time < len(text) and text[time].isdigit():
                    event.asks.append(int(text[time]))
                    time += 1
                break
            else:
                if text[time] == 'time':
                    event.time = text[time + 1]
                elif text[time] == "time_us":
                    event.timeUS = int(text[time + 1])
                elif text[time] == "best_bid_price":
                    event.bbp = int(text[time + 1])
                elif text[time] == "spread":
                    event.spread = int(text[time + 1])
                elif text[time] == "best_ask_price":
                    event.bap = int(text[time + 1])
                time += 2
    elif text[time] == 'deal':
        event = Event('deal')
        time += 1
        
        while time < len(text) and text[time] != 'book' and text[time] != 'deal':
            if text[time] == 'time':
                event.time = text[time + 1]
            elif text[time] == "time_us":
                event.timeUS = int(text[time + 1])
            elif text[time] == "direction":
                event.direction = text[time + 1]
            elif text[time] == "price":
                event.price = int(text[time + 1])
            elif text[time] == "volume":
                event.volume = int(text[time + 1])
            elif text[time] == "number_of_trades_in_batch":
                event.ntb = int(text[time + 1])
            time += 2
    return event

In [3]:
import numpy as np

# работа с одномерными временными рядами
class TimeSeries:
    def __init__(self, name):
        self.name = name
        self.data = np.array([])
        self.time = np.array([])
    
    # добавление элемента
    def add(self, time, val):
        self.data = np.append(self.data, val)
        self.time = np.append(self.time, time)
        
        if len(self.data) >= 3:
            if self.data[-2] > self.data[-1] and self.data[-2] > self.data[-3]:
                self.last_local_max = self.time[-2]
            elif self.data[-2] < self.data[-1] and self.data[-2] < self.data[-3]:
                self.last_local_min = self.time[-2]
        else:
            self.last_local_max = self.time[0]
            self.last_local_min = self.time[0]
        return self
    
    # подсчёт средних
    def mean(self):
        return self.data.mean()    
    def meanInTime(self, delta):
        return self.data[self.time > self.time[-1] - delta].mean()
    
    # выделение признаков
    def addFeatures(self, features, time):
        if len(self.data) != 0:        
            features[self.name + "_time_to_last_local_max"][time] = self.time[-1] - self.last_local_max
            features[self.name + "_time_to_last_local_min"][time] = self.time[-1] - self.last_local_min
        
            features[self.name + "_mean_0.1sec"][time] = self.meanInTime(100) 
            features[self.name + "_mean_0.3sec"][time] = self.meanInTime(300)
            features[self.name + "_mean_1sec"][time] = self.meanInTime(1000)
            features[self.name + "_mean_3sec"][time] = self.meanInTime(3000)
            features[self.name + "_mean_10sec"][time] = self.meanInTime(10000)
            features[self.name + "_mean_100sec"][time] = self.meanInTime(100000)
            features[self.name + "_mean_1000sec"][time] = self.meanInTime(1000000)
            features[self.name + "_mean_10000sec"][time] = self.meanInTime(10000000)      
            #features[self.name + "_mean (full)"][time] = self.mean()

In [4]:
# хранение текущего состояния и истории
class State:
    def __init__(self, event):
        self.bids = np.array(event.bids)
        self.asks = np.array(event.asks)
        self.time = event.time
        self.timeUS = event.timeUS
        self.bbp = event.bbp
        self.bap = event.bap
        self.bbp_history = TimeSeries("bbp")
        self.bap_history = TimeSeries("bap")
        self.bids_history = TimeSeries("bids")
        self.asks_history = TimeSeries("asks")
        self.sells_history = TimeSeries("sells")
        self.buys_history = TimeSeries("buys")
        
    def addFeatures(self, features):
        if len(self.sells_history.data) == 0:
            return
        
        features["bbp"][self.time] = self.bbp
        features["bap"][self.time] = self.bap
        features["num_of_bbp"][self.time] = self.bids[0]
        features["num_of_bap"][self.time] = self.asks[0]
        features["average_bid"][self.time] = self.bids_history.data[-1]
        features["average_ask"][self.time] = self.asks_history.data[-1]
        self.bbp_history.addFeatures(features, self.time)
        self.bap_history.addFeatures(features, self.time)
        self.bids_history.addFeatures(features, self.time)
        self.asks_history.addFeatures(features, self.time)
        self.buys_history.addFeatures(features, self.time)
        self.sells_history.addFeatures(features, self.time)
        
    def processEvent(self, event):
        if event.name == "the end":
            return 0
        if event.name == "deal":
            if event.direction == "BUY":
                self.buys_history.add(event.timeUS // 1000, event.volume)
            else:
                self.sells_history.add(event.timeUS // 1000, event.volume)
            return -1
        else:
            self.bbp_history.add(self.timeUS // 1000, self.bbp)
            self.bap_history.add(self.timeUS // 1000, self.bap)
            self.bids_history.add(self.timeUS // 1000, self.bids.dot(np.arange(self.bbp - 9, self.bbp + 1)[::-1]) / self.bids.sum())
            self.asks_history.add(self.timeUS // 1000, self.asks.dot(np.arange(self.bap, self.bap + 10)) / self.asks.sum())
            
            self.bids = np.array(event.bids)
            self.asks = np.array(event.asks)
            self.time = event.time
            self.timeUS = event.timeUS
            self.bbp = event.bbp
            self.bap = event.bap
            return 1

# Задача 1A

### Описательные характеристики:
Чтобы чуть обобщить задачу, будем рассматривать одномерные временные ряды - например, временной ряд лучшей цены на бидах. Из одномерного временного ряда можно выделять признаки независимо от физического смысла самого ряда:

1. среднее за некоторый промежуток времени. Имеет смысл взять несколько промежутков времени. Поскольку информация в далёком прошлом не столь важна, можно взять их по логарифмической шкале - за 1/10 секунду, за последнюю секунду, за 10 секунд, за 100 секунд, и так далее.

2. помимо среднего, можно брать и другие агрегирующие значения: минимум, максимум, медиана или другие квантили, дисперсия.

3. время с последнего локального максимума / локального минимума.

4. среднее расстояние между локальными экстремумами

5. угол наклона аппроксимирующей прямой. Под аппроксимирующей прямой подразумевается прямая, построенная по имеющимся точкам время-значение методом, например, наименьших квадратов.

6. прошедшее время с момента, когда значение ряда было таким же, как текущее. Можно также рассмотреть, какие значения принимал ряд в прошлом после такого же значения. При этом для статичных промежутков таким "предыдущим моментом" можно считать как предыдущий момент времени, так и требовать, чтобы между ним и текущим моментом встречались и другие значения. Продолжительность статичного промежутка (то есть сколько держится текущее значение) тоже можно считать признаком. 


Временные ряды, которые можно рассмотреть:
1. Лучшая цена на бидах/асках

2. Средняя цена на бидах/асках среди лучших 10 (поскольку только лучшие 10 доступны...), с учётом объёма заявок

3. Для рядов 1-2 можно рассмотреть их дискретные производные (разницы) первого-второго порядка

4. Количество продаж/покупок, объём агрессивных продаж/покупок

Также можно "копировать" признаки, полученные для предыдущих $N$ моментов времени.

In [5]:
load("2017.04.25")   # загружаем файл
time = 0             # "время" - индекс в списке слов файла, для парсинга
state = State(NextEvent())  # инициализируем состояние первой книгой из файла.

In [6]:
from collections import defaultdict

features = defaultdict(lambda: {}, {})
status = -1                   
while state.time[1] != '2':           # соберём информацию до 12:00
    if status == 1:                   # будем собирать данные в моменты времени, когда в файле записаны книги
        state.addFeatures(features)
    status = state.processEvent(NextEvent())  # обрабатываем следующее событие. Если это сделка, status будет равен -1

# Задача 1B

### Сигналы:

Из каких соображений возможна прибыльность сигналов? Если есть некоторый алгоритм предсказывания будущего с точностью, хоть и близкой к подбрасыванию монетки, но достаточной, чтобы компенсировать разницу на спреде, возможно проводить большое количество сделок и в среднем получать ненулевую прибыль. Предсказывание будущего при этом должно проводиться на некоторый промежуток времени вперёд; поскольку в задании далее предполагалось закрывать сделки через секунду (иначе говоря, через фиксированное время), то предсказателю достаточно пытаться вычислить моменты, в которые лучшая цена на бидах была бы выше, чем лучшая цена на асках через указанное время. Такие моменты будут моментами, в которые сигнал будет говорить "продавай". Аналогично можно настроить генератор команд "покупай". 

В качестве такого предсказателя на основе какого-то подмножества вышеописанных признаков может выступать любой алгоритм машинного обучения. Например, исходя из интепретируемой идеи искать моменты времени в прошлом, в которые "ситуация последних миллисекунд" была похожей, можно пользоваться kNN-ом (который к тому же онлайновый и его понятно как встраивать непосредственно в живую торговлю). kNN, правда, потребует предобработки признаков (точнее - введения аккуратно построенной функции расстояния между двумя прецендентами). Можно обучить более сложную модель, градиентный бустинг или случайный лес, которые по сути сами ищут паттерны в признаковом описании данных и, если какие-то зависимости там есть, позволят узнавать о будущем дополнительную информацию. В задаче 2 я придерживался такого подхода.

Однако, судя по оценке качества полученного алгоритма, такой подход имеет принципиальный недостаток. Он предполагает, что в зависимости от истории и текущего состояния возможно предсказывать дальнейшее движение цены. Пытаться избежать этой проблемы можно, пробуя строить сигналы на основе других целевых переменных, то есть дать другое определение момента времени, в который стоит выдавать сигнал о покупке или продаже. Иначе говоря, в предыдущем подходе алгоритм машинного обучения искал моменты, через секунду после которых цена значительно (более, чем на размер спреда) выросла/упала. Можно пробовать искать псевдоглобальные экстремумы, то есть моменты, после которых цена значительно долгое время не опускается/поднимается ниже текущего уровня, или искать более глобальные характеристики ряда вроде прогнозирования дальних промежутков времени для использования в качестве новых признаков. 

Тем не менее, стоит учитывать, что при случайной торговле (а попытки торговать, при помощи монетки предсказывая будущее - это как раз такой случай) в среднем результат будет убыток в размере спреда на лот. Поэтому стоит задуматься о торговле не в формате "купить, а потом продать через секунду"/"продать, а потом купить через секунду", а рассматривая схемы, зависящее от предыдущих покупок, без фиксированного времени закрытия сделки. Банальный вариант: купим одну акцию. Если цена лучшего бида вырастет до цены на один рубль больше, то стоит тут же совершить продажу и получить прибыль. Ничего не выйдет, только если цена никогда не выйдет на нужный уровень, что маловероятно, но возможно и может принести огромный убыток, причём без фиксированного времени закрытия действовать в таком случае будет сложно.

Усложнённая версия того же чисто теоретического размышления. Совершим и покупку и продажу (казалось бы, это не имеет никакого смысла). В каждый момент времени будем иметь только одну "открытую" сделку на покупку и одну "открытую" сделку на продажу, которые будут определять наш коридор: как только графики лучших цен на асках и бидах выходят за пределы этого коридора, мы закрываем соответствующую сделку, получаем с неё прибыль в один доллар и переоткрываем с новой ценой (если закрылась "на покупку", открываем "на покупку", если "на продажу" - то "на продажу"). "Размер" коридора - расстояние между ценами, по которым мы совершили текущую покупку и текущую продажу - изначально был равен спреду, обозначим его $S$ и будем предполагать константным. После закрытия и переоткрытия сделки коридор расширяется на величину $S$ + 1. После $K$ итераций прибыль составит $K$ рублей, а размер коридора - $S + (S + 1)K$. Поскольку всегда открыто ровно две сделки, это можно считать исходным константным вложением, которое, однако, не скоро окупится в силу вечного расширения коридора и всё меньшей вероятности того, что графики пересекут одну из границ. Зато теоретически нет риска бесконечных потерь и некая гарантия прибыли за счёт неразумно большого времени её ожидания.

Возможно, такую стратегию можно оптимизировать; в любой момент времени можно оценить вероятность того, что график цен пересечёт одну из его границ, исходя из предположения, что он с равной вероятностью пойдёт как вверх, так и вниз на каждом шаге, и что изменение цены происходит раз в $t$ секунд, которые можно оценить по истории. Если график оказывается в середине коридора, коридор можно за счёт убытка сузить, сокращая время ожидания.

Ещё небольшое размышление. Раз движение цены совершенно случайно, то возможно, стоит пытаться извлечь информацию не из динамики цен. Известно, что цена определяется поведением других игроков на рынке, и известно, что их целью в целом также является прибыль. Возможно, можно моделировать их действия, оценив по количеству совершённых сделок количество игроков, и настроив такое количество торговых базовых алгоритмов вроде основанных на машинном обучении или даже более простых правил по истории совершённых сделок. Это кажется ещё более трудоёмкой задачей, но при достаточном приближении можно получать некоторые более информативные прогнозы на будущее. С другой стороны, моделирование действий игрока проще (особенно если известны виды использующихся алгоритмов), чем моделирование результата обработки действий многих игроков, но усложняется ограниченностью имеющейся информации, так как неизвестно, какой именно игрок совершил какое действие и сколько игроков на самом деле.

In [7]:
import pandas as pd
DF = pd.DataFrame(features)
DF

,asks_mean_0.1sec,asks_mean_0.3sec,asks_mean_10000sec,asks_mean_1000sec,asks_mean_100sec,asks_mean_10sec,asks_mean_1sec,asks_mean_3sec,asks_time_to_last_local_max,asks_time_to_last_local_min,...,sells_mean_0.1sec,sells_mean_0.3sec,sells_mean_10000sec,sells_mean_1000sec,sells_mean_100sec,sells_mean_10sec,sells_mean_1sec,sells_mean_3sec,sells_time_to_last_local_max,sells_time_to_last_local_min
10:00:20.068,56886.953615,56886.953615,56886.953615,56886.953615,56886.953615,56886.953615,56886.953615,56886.953615,74.0,74.0,...,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,0.0,0.0
10:00:20.084,56886.952723,56886.952723,56886.952723,56886.952723,56886.952723,56886.952723,56886.952723,56886.952723,75.0,75.0,...,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,0.0,0.0
10:00:20.093,56886.952053,56886.952053,56886.952053,56886.952053,56886.952053,56886.952053,56886.952053,56886.952053,91.0,91.0,...,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,0.0,0.0
10:00:20.307,56886.947368,56886.951533,56886.951533,56886.951533,56886.951533,56886.951533,56886.951533,56886.951533,100.0,100.0,...,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,0.0,0.0
10:00:20.310,56888.581250,56887.128911,56887.114505,56887.114505,56887.114505,56887.114505,56887.114505,56887.114505,314.0,314.0,...,2.000000,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,241.0,241.0
10:00:20.313,56887.542824,56887.066460,56887.059040,56887.059040,56887.059040,56887.059040,56887.059040,56887.059040,3.0,317.0,...,2.000000,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,241.0,241.0
10:00:20.314,56887.181160,56887.011130,56887.008940,56887.008940,56887.008940,56887.008940,56887.008940,56887.008940,6.0,320.0,...,2.000000,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,241.0,241.0
10:00:20.324,56887.000328,56886.965022,56886.966547,56886.966547,56886.966547,56886.966547,56886.966547,56886.966547,7.0,321.0,...,2.000000,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,241.0,241.0
10:00:20.337,56886.891829,56886.926007,56886.930210,56886.930210,56886.930210,56886.930210,56886.930210,56886.930210,17.0,331.0,...,3.437500,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,2.0,2.0
10:00:20.339,56886.744259,56886.845814,56886.868623,56886.868623,56886.868623,56886.868623,56886.868623,56886.868623,30.0,344.0,...,3.437500,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,2.0,2.0


In [ ]:
DF.dropna(inplace=True)  # сбрасываем первые преценденты до начала агрессивных покупок и продаж

# используем признаки с двух предыдущих "шагов".
def stack(DF, N):
    modDF = pd.DataFrame(index=DF[N-1:].index)
    for i in range(N):
        newDF = DF.copy()[i:len(DF)-N+i+1]
        newDF.index = modDF.index        
        modDF = pd.concat([modDF, newDF], axis=1)
    return modDF
DF = stack(DF, 3)

In [10]:
from sklearn.ensemble import RandomForestClassifier as RF

# обучаем случайный лес предсказывать удачные момент для продажи на основе цены через 30 шагов
STEPS = 30
y = DF["bap"][STEPS:-1].as_matrix()[:, 0] < DF["bbp"][:-STEPS-1].as_matrix()[:, 0]
want_to_sell_model = RF().fit(DF[:200000], y[:200000])
predict = want_to_sell_model.predict(DF[200000:-STEPS-1])

In [11]:
from sklearn.metrics import roc_auc_score as aucroc
# результат по auc-roc-у выдаёт от 0.49 до 0.53, что в среднем выше 0.5 и, казалось бы, вселяет надежду
aucroc(y[200000:], predict)

0.51498805606683018

In [12]:
# аналогично строим модель, предсказывающую удачные моменты для покупок
y = DF["bbp"][STEPS:-1].as_matrix()[:, 0] > DF["bap"][:-STEPS-1].as_matrix()[:, 0]
want_to_buy_model = RF().fit(DF[:200000], y[:200000])
predict = want_to_buy_model.predict(DF[200000:-STEPS-1])

aucroc(y[200000:], predict)

0.51250713887289467

# Задача 2

В качестве сигнала был взят случайный лес на признаках 1, 3 из временных рядов 1, 2, 4, указанных в задаче 1а, а также на копиях этих признаков с двух предыдущих шагов. Шагом я называю моменты времени, когда в данных предоставляется информация о стаканах (book). Целевой переменной для модели, ищущей моменты для продаж, является отрицательность разницы между ценой на покупках через 30 шагов и текущей ценой на продажах.

Обучение проводилось на первых 200000 шагах, оценка прибыльности проводится на следующих 2000 секундах по указанному в задании алгоритму - сделки "закрываются" через секунду после открытия.

In [13]:
# начинаем заново считывать из файла события; данные, на которых алгоритм обучался, пропускаем
time = 0
state = State(NextEvent())
while state.timeUS < 1493204834000000:
    status = state.processEvent(NextEvent())

In [15]:
# класс одной сделки. В result будет записан её исход, то есть прибыль/убыток от совершённого действия
class Deal:
    def __init__(self, volume, price, direction, timeUS):
        self.volume = volume
        self.price = price
        self.direction = direction
        self.timeUS = timeUS
        self.result = 0
    
    # закрытие сделки
    def execute(self, state):
        if self.direction == "buy":
            self.result = (state.bbp - self.price) * self.volume
        else:
            self.result = (-state.bap + self.price) * self.volume

# для удобства генерим на каждом шаге только текущие признаки (их ONE_BLOCK штук)
# и присоединяем признаки с предыдущих двух шагов
NUM_OF_FEATURES = len(DF.columns)
ONE_BLOCK = NUM_OF_FEATURES / 3
all_features = np.zeros((1, NUM_OF_FEATURES))

# функция сигнала - по состоянию возвращает предлагаемое действие
def Signal(state):
    features = defaultdict(lambda: {}, {})
    global all_features
    
    # собираем признаки
    state.addFeatures(features)
    # присоединяем признаки с предыдущих 2 шагов
    all_features[0][ONE_BLOCK:] = all_features[0][:NUM_OF_FEATURES - ONE_BLOCK]
    all_features[0][:ONE_BLOCK] = pd.DataFrame(features).as_matrix()
    
    # смотрим на выход моделей
    if want_to_buy_model.predict(all_features)[0]:
        return "buy"
    if want_to_sell_model.predict(all_features)[0]:
        return "sell"
    return ""

In [ ]:
my_deals = []  # список совершённых сделок
processed_deals = 0  # указатель на первую незакрытую сделку
all_features = np.zeros((1, NUM_OF_FEATURES))

while state.timeUS < 1493206834000000:  #торгуем 2000 секунд
    if status == 1:                     # если последнее событие - книга (book)
        signal = Signal(state)          # получаем сигнал
        if signal == "buy":             # совершаем сделки
            my_deals.append(Deal(min(20, state.asks[0]), state.bap, "buy", state.timeUS))
        if signal == "sell":
            my_deals.append(Deal(min(20, state.bids[0]), state.bbp, "sell", state.timeUS))
    
    # закрываем сделки, с момента совершения которых прошла секунда
    while processed_deals < len(my_deals) and state.timeUS - 1000000 > my_deals[processed_deals].timeUS:
        my_deals[processed_deals].execute(state)
        processed_deals += 1
        
    status = state.processEvent(NextEvent())

In [17]:
# считаем суммарную "прибыль"
summary = np.array([d.result for d in my_deals])
summary.sum()

-301296

Средний результат за лот: -1.79
Количество сделок: ~4.9 сделки в секунду
Стоит отметить, что это потому, что сигналу предлагается кричать "покупай!" в каждый момент времени, т.е. в каждый шаг. Если ситуация не сильно меняется, он продолжает выдавать тот же сигнал и покупает одни и те же лоты. На деле, если бы покупка была бы совершена, из-за агрессивной заявки ситуация бы сильно поменялась из-за действий самого сигнала. Однако в этой модели этого не учитывается.

Ещё стоит отметить, что за 2000 секунд направление сделок (покупать или продавать) менялось всего лишь 105 раз. То есть деятельность алгоритма довольно последовательная, какое-то существенное время (20 секунд в среднем!) он хочет продавать, затем уже покупать, и так далее. При этом есть существенные периоды, когда не было принято решения ни торговать, ни покупать, что на вид кажется логичным. В качестве небольшой проверки адекватности получившегося результата, ниже проверка првоедена ещё раз, но оценка качества алгоритма проводится по-другому: при покупках цена на асках сранивается с ценой на тех же асках через секунду, а при продажах - с ценой на тех же бидах через секунду. Это позволит понять, улавливаливает ли хоть сколько-то алгоритм какое-то движение цен, или AUC-ROC 0.51 вместо 0.5 совсем ничего не значит.  

In [ ]:
time = 0
state = State(NextEvent())
while state.timeUS < 1493204834000000:
    status = state.processEvent(NextEvent())
    
class Deal:
    def __init__(self, volume, price, direction, timeUS):
        self.volume = volume
        self.price = price
        self.direction = direction
        self.timeUS = timeUS
        self.result = 0
        
    def execute(self, state):
        if self.direction == "buy":
            self.result = (state.bap - self.price) * self.volume
        else:
            self.result = (-state.bbp + self.price) * self.volume
            
my_deals2 = []
processed_deals2 = 0
all_features = np.zeros((1, NUM_OF_FEATURES))

while state.timeUS < 1493206834000000:  #2000 сек примерно полчаса
    if status == 1:        
        signal = Signal(state)
        if signal == "buy":
            my_deals2.append(Deal(min(20, state.asks[0]), state.bap, "buy", state.timeUS))
        if signal == "sell":
            my_deals2.append(Deal(min(20, state.bids[0]), state.bbp, "sell", state.timeUS))
            
    while processed_deals2 < len(my_deals2) and state.timeUS - 1000000 > my_deals2[processed_deals2].timeUS:
        my_deals2[processed_deals2].execute(state)
        processed_deals2 += 1
        
    status = state.processEvent(NextEvent())

In [19]:
summary2 = np.array([d.result for d in my_deals2])
summary2.sum()

4377

Средний "результат" за лот - ~0.05, то есть какое-то движение цен таки улавливается, хоть этого и не хватает, чтобы превзойти разницу на спреде.